<a href="https://colab.research.google.com/github/davide1536/minesweeper/blob/main/mineSweeper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install python-sat[pblib,aiger]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import random 
import numpy as np
from pysat.solvers import Solver
from pysat.card import *

In [ ]:
def checkForVictory(board, knowledge):
  for row in range(SIZE):
    for col in range(SIZE):
      if board[codeId(row, col)].value == -1:
        if board[codeId(row, col)].flag == 0 or board[codeId(row,col)].safe == 1:
          print("trovata falsa mina")
          print(board[codeId(row, col)].id)
          print(board[codeId(row, col)].flag)
          print(board[codeId(row,col)].safe)
          return False
      else:
        if board[codeId(row, col)].flag == 1 or board[codeId(row,col)].safe == 0:
          print("trovata falsa safe")
          print(board[codeId(row, col)].id)
          print(board[codeId(row, col)].flag)
          print(board[codeId(row,col)].safe)
          return False
    return True

In [ ]:
def intersection(lst1, lst2):
  return list(set(lst1).intersection(lst2))

In [ ]:
def difference(list1, list2):
  return list(set(list1) - set(list2))

In [ ]:
def selectCell(board, knowledge, tried):
  #print("cerco cella")
  for cells in knowledge.totalCells:
    for cell in cells:
      if board[cell].safe != 1 and board[cell].flag != 1 and not(cell in tried):
        #print(tried)
        #print(cell)
        #print(board[cell].safe)
        #print(board[cell].flag)
        return [cell,3]

  for row in range(SIZE):
    for col in range(SIZE):
      if not(board[codeId(row,col)].id in cellSelected) and not(board[codeId(row,col)].id in mineDiscovered):
        return [board[codeId(row,col)].id, -3]

In [ ]:
def createClauses(board, knowledge):
  clauses = []
  for i,cells in enumerate(knowledge.totalCells):
    if knowledge.count[i] != 0 and knowledge.count[i] != len(cells) and len(cells) > 0:
      #print("transformo in clausola celle: ", cells)
      #print("count: ", knowledge.count[i])
      cnfClause = CardEnc.equals(lits = cells, bound = int(knowledge.count[i]), top_id=100 + i*10, encoding=1)
      clauses.append(cnfClause)

  return clauses


In [ ]:
def createAssumptions(board,knowledge):
  assumptions = []
  for cell in cellSelected:
    assumptions.append(board[cell].proposition)

  for mine in mineDiscovered:
    assumptions.append(board[mine].proposition)

  return assumptions

In [ ]:
def removePositiveProposition(knowledge, currentCells, currentIndex, dictionary):
  for i,cells in enumerate(knowledge.totalCells):
    if i != currentIndex:
      removeAndUpdateProposition(knowledge, currentCells, cells, dictionary, i)
      #print("differenza: ", knowledge.totalCells[i])
      if knowledge.totalCells[i] == []:
        #print("tolgo liste vuote")
        knowledge.count[i] = -1
        # del knowledge.totalCells[i]
        # del knowledge.count[i]

In [ ]:
def removeNegativeProposition(knowledge, currentCells, currentIndex, dictionary):
  for i,cells in enumerate(knowledge.totalCells):
    if i != currentIndex:
      numberOfCells = removeAndUpdateProposition(knowledge, currentCells, cells, dictionary,i)
      #print("differenza: ", knowledge.totalCells[i])
      #print("counter attuale: ",knowledge.count[i])
      knowledge.count[i] -= numberOfCells
      #print("counter aggiornato: ", knowledge.count[i])
      if knowledge.totalCells[i] == []:
       # print("tolgo lister vuote")
       knowledge.count[i] = -1
        # del knowledge.totalCells[i]
        # del knowledge.count[i]

In [ ]:
def removeAndUpdateProposition(knowledge, currentCells, cells, dictionary, index):

  #print("rimuovo proposizioni")
  intersect= intersection(cells, currentCells)
  #print("intersezione tra ", cells, " e ",currentCells, " é ", intersect)
  diff = difference(knowledge.totalCells[index], intersect)
  knowledge.totalCells[index] = diff
  return len(intersect)
 

In [ ]:
def clearKnowledge(knowledge, board):
  for i, cells in enumerate(knowledge.totalCells):
    if cells == [] and knowledge.count[i] == -1:
      del knowledge.totalCells[i]
      del knowledge.count[i]

In [ ]:
def updateKnowledge(knowledge,board):

  for i,cells in enumerate(knowledge.totalCells):
      if knowledge.count[i] == 0:
        #print("trovato celle con counter 0")
        for cell in cells:
          # if not(cell in cellSelected):
          #   cellSelected.append(cell)

          if board[cell].proposition > 0:
            board[cell].proposition = -board[cell].proposition
        removePositiveProposition(knowledge, cells, i, board)

  for i,cells in enumerate(knowledge.totalCells):  
      if knowledge.count[i] == len(cells):
        for cell in cells:
          if not(cell in mineDiscovered):
            mineDiscovered.append(cell)
          board[cell].safe = 0
          board[cell].flag = 1
          
        removeNegativeProposition(knowledge, cells, i,board)

In [ ]:
def buildCellDictionary(board):
  dictionary = {}
  for row in range(SIZE):
    for col in range(SIZE):
      dictionary[codeId(row, col)] = createCell(board[row,col], row, col)
  
  return dictionary


In [ ]:
def codeId(value1, value2):
  return value1 * SIZE + value2 + 1

In [ ]:
def findSafeCell(board, knowledge):
  print("cerco cella con inferenza")
  print("knowledge accumulata sino ad ora")
  for i,cells in enumerate(knowledge.totalCells):
    print(cells, " ", knowledge.count[i])
  clauses = []
  cellTried = []
  assumptions = createAssumptions(board, knowledge)
  print("assumption: ", assumptions)
  print("trovate ", len(assumptions), " assumptions")
  setOfFormulas = createClauses(board, knowledge)
  negativeSolution = ''
  positiveSolution = ''
  cell = 0
  while not(positiveSolution == False) and not(negativeSolution == False) and not(cell == None):


  
    cell,mode = selectCell(board,knowledge, cellTried)
    if mode == -3:
      return cell
    if not(cell == None):
      cellTried.append(cell)

      
      print("provo a selezionare la cella: ", cell)

      #print("con proposizione: ", board[cell].proposition)


      positiveSolver = Solver()
      negativeSolver = Solver()
      for formula in setOfFormulas:
        #print("clauses: ", formula.clauses)
        for clause in formula.clauses:
          #print("clause: ",clause)
          positiveSolver.add_clause(clause)
          negativeSolver.add_clause(clause)


      #positiveSolver.append_formula(formula=clauses)
      #print("proposition: ", int(-board[cell].proposition))
      negativeSolver.add_clause([int(board[cell].proposition)])

      
      #negativeSolver.append_formula(clauses, no_return=True)
      positiveSolver.add_clause([-int(board[cell].proposition)])

      negativeSolution = negativeSolver.solve(assumptions=assumptions)
      positiveSolution = positiveSolver.solve(assumptions = assumptions)

      print(negativeSolution)
      print(positiveSolution)

      negativeSolver.delete()
      positiveSolver.delete()

  if negativeSolution == False:
    print("ho trovato una cella safe ",cell)
    return cell
  elif positiveSolution == False:
    print("ho trovato una mina")
    createKnowledge(board[cell], board, knowledge)
    return -2
  else:
    return -1


In [ ]:
def createKnowledge(cell, board, knowledge):
  #print("creo knowledge della cella in posizione: ", cell.row, " ", cell.col)
  currentKnowledge = []
  if cell.value == -1:
    currentKnowledge.append(cell.id)
    knowledge.count.append(1)
  else:

    currentKnowledge.append(cell.id)
    knowledge.totalCells.append(currentKnowledge)
    knowledge.count.append(0)

    currentKnowledge = []
    
    value = cell.value
    for i in range(3):
      for j in range(3):
        if (cell.row-1+i >= 0 and cell.row-1+i <= SIZE-1) and (cell.col-1+j >= 0 and cell.col-1+j <= SIZE-1):
          if cell.id != board[codeId(cell.row-1+i,cell.col-1+j)].id:
              
              currentKnowledge.append(board[codeId(cell.row-1+i,cell.col-1+j)].id)
  
    knowledge.count.append(value)

  knowledge.totalCells.append(currentKnowledge)
  # print("knowledge accumulata sino ad ora")
  # for i,cells in enumerate(knowledge.totalCells):
  #   print(cells, " ", knowledge.count[i])
  updateKnowledge(knowledge,board)
  clearKnowledge(knowledge, board)

  # print("knowledge aggiornata")
  # for i,cells in enumerate(knowledge.totalCells):
  #   print(cells, " ", knowledge.count[i])
  # print("dopo l'aggiornamento: ")
  # for i,cells in enumerate(knowledge.totalCells):
  #   print(cells, " ", knowledge.count[i])


In [ ]:
def checkTrivialSafeCell(board, knowledge):
  for i,cells in enumerate(knowledge.totalCells):
    if knowledge.count[i] == 0:
      
      for cell in cells:
        if board[cell].selected == 0:
          return cell
  
  return -1

In [ ]:
def selectNextSafeCell(board, knowledge):
  
  nextCell = checkTrivialSafeCell(board, knowledge, )
  
  #if there aren't any trivial safe cell use sat solver on clause
 
  if nextCell == -1:
    nextCell = findSafeCell(board, knowledge)

  #if i've found a mine thanks to the solver i'll try to reuse the trivial solver
  if nextCell == -2:
    nextCell = checkTrivialSafeCell(board, knowledge)


  if nextCell == -1:
    result = nextCell
  else:
    result = board[nextCell]
    
  return result


In [ ]:
def countMines(row,col,board, size):
  counter = 0

  for i in range(3):
    for j in range(3):
      if (row-1+i >= 0 and row-1+i <= size-1) and (col-1+j >= 0 and col-1+j <= size-1):
        if board[row-1+i, col-1+j] == -1:
          counter += 1
        
     
  
  return counter

In [ ]:
#function that, for each cell, counts the number of mines around it
def fillBoard(board, size):
  
  for row in range(size):
    for col in range(size):
      if board[row,col] != -1:
        n_mines = countMines(row, col, board, size)
        board[row,col] = n_mines
    
      
    
  return board


In [ ]:

def buildBoard(size, probability):
  total_mines = 0
  #board[i][j] = 0 --> no bomb
  #board[i][j] = 1 --> bomb
  print("inizializzo board di gioco")
  board = np.zeros((size,size))
  for row in range(size):
    for col in range(size):
      randomNumber = random.uniform(0,1)
      if randomNumber <= probability:
        board[row,col] = -1
        total_mines += 1
      else:
        board[row,col] = 0


  board = fillBoard(board, size)
  print("board di gioco inizializzata con ",total_mines, " ","mine")
  return board
      

In [ ]:
def createCell(value, row, col):
  cell = Cell()
  cell.value = int(value)
  
  cell.safe = 0
    
  cell.selected = 0
  cell.flag = 0
  cell.row = row
  cell.col = col
  cell.proposition = int(codeId(row,col))
  cell.id = int(codeId(row,col))
  return cell

In [ ]:
def startGame(board, knowledge):
  result = ""
  firstCell = board[codeId(0,0)]
  firstCell.selected = 1
  cellSelected.append(firstCell.id)

  if firstCell.value == -1:
    result = "lose"
  else:
    createKnowledge(firstCell, board, knowledge)

 

  while not(result == "win") and not(result == "lose"):
    
    cell = selectNextSafeCell(board, knowledge)
    if cell == -1:
      return "could not find a safe cell"
    
    elif cell == 1:
      result = "win"

    else:
      print("ho selezionato la cella ", cell.id)
      cell.selected = 1
      cell.safe = 1
      if not(cell.id in cellSelected):
        cellSelected.append(cell.id)
    
      if len(cellSelected) + len(mineDiscovered) == SIZE * SIZE:
        victory = checkForVictory(board, knowledge)
        if victory:
          result = "win"
        else:
          return "An error occured"
      if cell.value == -1:
        result = "lose"
      else:
        createKnowledge(cell, board, knowledge)


  return result

In [ ]:
class InternalRepresentation:
  def __init__(self) -> None:
    self.totalCells = []
    self.count = []

In [ ]:
class Cell:
  def __init__(self) -> None:
    self.safe = 0
    self.value = 0
    self.proposition = 0
    self.id = 0
    self.selected = 0
    self.row = 0
    self.col = 0
    self.flag = 0

In [ ]:
#build a 9x9 board
SIZE = 9
bomb_probability = 1/6
board = buildBoard(SIZE, bomb_probability)
print(board)

inizializzo board di gioco
board di gioco inizializzata con  12   mine
[[ 1.  2.  3. -1. -1.  3. -1.  2. -1.]
 [ 1. -1. -1.  3.  3. -1.  2.  2.  1.]
 [ 1.  2.  2.  1.  1.  1.  1.  0.  0.]
 [ 0.  0.  1.  1.  1.  0.  0.  0.  0.]
 [ 1.  1.  1. -1.  1.  0.  0.  1.  1.]
 [-1.  1.  2.  2.  2.  0.  0.  1. -1.]
 [ 1.  1.  1. -1.  1.  0.  0.  1.  1.]
 [ 1.  1.  1.  1.  1.  0.  0.  0.  0.]
 [-1.  1.  0.  0.  0.  0.  0.  0.  0.]]


In [ ]:
#start game from the up-left corner
knowledge = InternalRepresentation();
cellSelected = []
mineDiscovered = []
boardDictionary = buildCellDictionary(board)
result = startGame(boardDictionary, knowledge)
print("risultato: ", result);      

cerco cella con inferenza
knowledge accumulata sino ad ora
[1]   0
[10, 2, 11]   1
assumption:  [-1]
trovate  1  assumptions
provo a selezionare la cella:  10
True
True
provo a selezionare la cella:  2
True
True
provo a selezionare la cella:  11
True
True
ho selezionato la cella  2
cerco cella con inferenza
knowledge accumulata sino ad ora
[1]   0
[10, 11]   1
[2]   0
[3, 10, 11, 12]   2
assumption:  [-1, -2]
trovate  2  assumptions
provo a selezionare la cella:  10
True
True
provo a selezionare la cella:  11
True
True
provo a selezionare la cella:  3
True
True
provo a selezionare la cella:  12
True
True
ho selezionato la cella  3
cerco cella con inferenza
knowledge accumulata sino ad ora
[1]   0
[10, 11]   1
[2]   0
[10, 11, 12]   2
[3]   0
[12, 11, 4, 13]   3
assumption:  [-1, -2, -3]
trovate  3  assumptions
provo a selezionare la cella:  10
True
True
provo a selezionare la cella:  11
True
True
provo a selezionare la cella:  12
True
False
ho trovato una mina
cerco cella con inferenza

AttributeError: ignored